In [1]:
import pandas as pd
from collections import defaultdict

In [2]:
fallliste = pd.read_csv("Fallliste_roh.csv",sep=";",skiprows=1,encoding="latin")

In [3]:
fallliste

,HDL_Fallnr,Fallnummer,MA,Kommentar,No 01,No 02,No 03,No 04,zu No 01,zu No 02,...,Unnamed: 62,Unnamed: 63,Unnamed: 64,Unnamed: 65,Unnamed: 66,Unnamed: 67,Unnamed: 68,Unnamed: 69,Unnamed: 70,2 Felder: Abrechnung-bis fuer DRG und Zu- oder Abschlag


In [4]:
fallliste.columns

Index(['HDL_Fallnr', 'Fallnummer', 'MA', 'Kommentar', 'No 01', 'No 02',
       'No 03', 'No 04', 'zu No 01', 'zu No 02', 'zu No 03', 'zu No 04', 'FZ',
       'SZ', 'MDK', 'MDK-Prüfgrund', 'DRG neu (aus P21-Regel)',
       'DRG alt (aus Entgelte)', 'BWR neu', 'BWR alt (aus SIM-pur)',
       'Diff BWR', 'MDC Hauptdiagnosegruppe', 'PCCL', 'Gruppierungsstatus',
       'Unnamed: 24', 'Unnamed: 25', 'Unnamed: 26',
       'untere Grenzverweildauer nach Katalog',
       'durchschnittliche Verweildauer nach Katalog',
       'Grenzverweildauer nach Katalog', 'Errechnete Verweildauer',
       'Kostengewicht fuer 10 Entgelte bei DRG: Feld 1 = Kostengwicht der DRG',
       'Unnamed: 32', 'Unnamed: 33', 'Unnamed: 34', 'Unnamed: 35',
       'Unnamed: 36', 'Unnamed: 37', 'Unnamed: 38', 'Unnamed: 39',
       'Unnamed: 40',
       'Entgeltschluessel fuer 10 Entgelte bei DRG: DRG und Zu- oder Abschlag in den er',
       'Unnamed: 42', 'Unnamed: 43', 'Unnamed: 44', 'Unnamed: 45',
       'Unnamed: 46', 'Un

In [5]:
fpp_lab = pd.read_csv("mibi/mibi_test_synonyme/Fallnummern_pro_Pruefgruende.csv",encoding="latin1",sep=";", dtype=str)
fpp_mibi = pd.read_csv("labor/Fallnr_pro_Pruefgrund.csv",encoding="latin1",sep=";")


In [5]:
quellen = ["MIBI"]
quellen_dateien = ["mibi/mibi_test_synonyme/textsuche_ergebnisse_2019-03-06_19-02-43.csv"]
quelle_lab = pd.read_csv("labor/Labor_2018_mit_Pruefgrund.csv",encoding="latin1",sep=";")

quellen_dfs =  [pd.read_csv(q,encoding="latin1",sep=";") for q in quellen_dateien]
quellen_dfs.append(quelle_lab)
quellen_dfs[0].rename(columns={"Fallnr.":"Fallnummer"},index=str,inplace=True)
quellen_dfs[1].rename(columns={"KH-ID":"Fallnummer"},index=str,inplace=True)

In [6]:
print(len(quellen_dfs[1]))
quellen_dfs[1] = quellen_dfs[1].loc[quellen_dfs[1]["Info zu Prüfgrund"].notnull()]
print(len(quellen_dfs[1]))

1409637
54926


In [7]:
lab = quellen_dfs[1].copy()
mibi = quellen_dfs[0].copy()
lab.head()

,Fallnummer,Laborkürzel,Messwert,Masseinheit,Referenzbereich,Datum,alt. Messwert,alt. Masseinheit,Prüfgrund,Info zu Prüfgrund
38,2317025683,Glucose,14.15,mmol/l,73-107,05.01.18 08:18:00,255.00,NaN,E11.91,14.15mmol/l > 5.0mmol/l
42,2317025683,Glucose,14.87,mmol/l,73-107,08.01.18 11:15:00,268.00,NaN,E11.91,14.87mmol/l > 5.0mmol/l
48,2317025683,Kalium,5.84,mmol/l,3.5-5.1,08.01.18 11:15:00,5.84,meq/l,E87.5,5.84mmol/l > 5.5mmol/l
52,2317025683,Glucose,7.27,mmol/l,73-107,10.01.18 10:16:00,131.00,NaN,E11.91,7.27mmol/l > 5.0mmol/l
83,2317025683,Glucose,10.43,mmol/l,73-107,12.01.18 16:21:00,188.00,NaN,E11.91,10.43mmol/l > 5.0mmol/l


In [159]:
lab =lab.groupby(['Fallnummer',"Prüfgrund"])["Info zu Prüfgrund"].apply(', '.join).reset_index()
#lab["Info zu Prüfgrund2"] = lab["Prüfgrund"].astype(str) +": " + lab["Info zu Prüfgrund"].astype(str)

lab.head()

,Fallnummer,Prüfgrund,Info zu Prüfgrund
0,2317025683,E11.91,"14.15mmol/l > 5.0mmol/l, 14.87mmol/l > 5.0mmol..."
1,2317025683,E87.5,5.84mmol/l > 5.5mmol/l
2,2317035801,D62,5.95mmol/l < 6.0mmol/l
3,2317035801,E11.91,"5.55mmol/l > 5.0mmol/l, 5.27mmol/l > 5.0mmol/l..."
4,2317035801,E87.6,2.93mmol/l < 3.5mmol/l


In [27]:
lab2=lab.groupby("Fallnummer")["Prüfgrund"].apply(lambda x: [str(x)]).apply("|||".join)

In [30]:
lab2.head()

Fallnummer
2317025683    38    E11.91\n42    E11.91\n48     E87.5\n52  ...
2317035801    86     E11.91\n93      E87.6\n95      N18.3\n1...
2317035963    150    E87.6\n159    E87.6\n174    E87.1\n191 ...
2317038555        217    N18.3\nName: 2317038555, dtype: object
2317039149          258    D62\nName: 2317039149, dtype: object
Name: Prüfgrund, dtype: object

In [172]:
#lab2 = lab.merge(lab, how="left",on="Fallnummer", suffixes=[" 1"," 2"])
#lab2.groupby("Fallnummer").head()

,Fallnummer,Prüfgrund 1,Info zu Prüfgrund 1,Prüfgrund 2,Info zu Prüfgrund 2
0,2317025683,E11.91,"14.15mmol/l > 5.0mmol/l, 14.87mmol/l > 5.0mmol...",E11.91,"14.15mmol/l > 5.0mmol/l, 14.87mmol/l > 5.0mmol..."
1,2317025683,E11.91,"14.15mmol/l > 5.0mmol/l, 14.87mmol/l > 5.0mmol...",E87.5,5.84mmol/l > 5.5mmol/l
2,2317025683,E87.5,5.84mmol/l > 5.5mmol/l,E11.91,"14.15mmol/l > 5.0mmol/l, 14.87mmol/l > 5.0mmol..."
3,2317025683,E87.5,5.84mmol/l > 5.5mmol/l,E87.5,5.84mmol/l > 5.5mmol/l
4,2317035801,D62,5.95mmol/l < 6.0mmol/l,D62,5.95mmol/l < 6.0mmol/l
5,2317035801,D62,5.95mmol/l < 6.0mmol/l,E11.91,"5.55mmol/l > 5.0mmol/l, 5.27mmol/l > 5.0mmol/l..."
6,2317035801,D62,5.95mmol/l < 6.0mmol/l,E87.6,2.93mmol/l < 3.5mmol/l
7,2317035801,D62,5.95mmol/l < 6.0mmol/l,N18.3,31.0ml/min im Intervall 30-59ml/min
8,2317035801,E11.91,"5.55mmol/l > 5.0mmol/l, 5.27mmol/l > 5.0mmol/l...",D62,5.95mmol/l < 6.0mmol/l
20,2317035963,E87.1,128.0mmol/l < 130.0mmol/l,E87.1,128.0mmol/l < 130.0mmol/l


In [155]:
lab.head()

,Fallnummer,Prüfgrund,Info zu Prüfgrund,Info zu Prüfgrund2
0,2317025683,E11.91,"14.15mmol/l > 5.0mmol/l, 14.87mmol/l > 5.0mmol...","E11.91: 14.15mmol/l > 5.0mmol/l, 14.87mmol/l >..."
1,2317025683,E87.5,5.84mmol/l > 5.5mmol/l,E87.5: 5.84mmol/l > 5.5mmol/l
2,2317035801,D62,5.95mmol/l < 6.0mmol/l,D62: 5.95mmol/l < 6.0mmol/l
3,2317035801,E11.91,"5.55mmol/l > 5.0mmol/l, 5.27mmol/l > 5.0mmol/l...","E11.91: 5.55mmol/l > 5.0mmol/l, 5.27mmol/l > 5..."
4,2317035801,E87.6,2.93mmol/l < 3.5mmol/l,E87.6: 2.93mmol/l < 3.5mmol/l


In [21]:
lab = lab.groupby('Fallnummer')["Prüfgrund"].apply(', '.join).reset_index()
lab.head()

TypeError: sequence item 14: expected str instance, float found

In [126]:
mibi.head()

,Fallnummer,ICD,ICD Klarname,Fundstelle im Text,ganzer Befund
0,2318008658,B95.91,actinomyces,. mäßig actinomyces turicensis\r\r\nbemerkung...,anaerobius\r\r\n (Anaerobier)\r\r\n...
1,2318011354,B96.7,clostridium,v\r\r\n clostridium difficile toxin a/b\r\r\n...,V\r\r\n Clostridium difficile Toxin A/B\r\r\n...
2,2318013728,B96.2,yersinia,yersinia,Yersinia negativ\r\r\n Campylobacter negativ...
3,2318013237,B96.7,clostridium,chweis von clostridium\r\r\n difficile\r\r\n,chweis von Clostridium\r\r\n difficile\r\r\n
4,2318021546,B96.0,mycoplasma,1.mycoplasma hominis negativ\r\r\n 2.ureaplasma.,von Hefen\r\r\nMYCOPLASMENKULTUR:\r\r\n 1.Myc...


In [100]:
alle = lab.merge(mibi,how="outer",on="Fallnummer",suffixes=["","_1"])
alle.head()

,Fallnummer,Laborkürzel,Messwert,Masseinheit,Referenzbereich,Datum,alt. Messwert,alt. Masseinheit,Prüfgrund,Info zu Prüfgrund,ICD,ICD Klarname,Fundstelle im Text,ganzer Befund
0,2317025683,Glucose,14.15,mmol/l,73-107,05.01.18 08:18:00,255.00,NaN,E11.91,14.15mmol/l > 5.0mmol/l,NaN,NaN,NaN,NaN
1,2317025683,Glucose,14.87,mmol/l,73-107,08.01.18 11:15:00,268.00,NaN,E11.91,14.87mmol/l > 5.0mmol/l,NaN,NaN,NaN,NaN
2,2317025683,Kalium,5.84,mmol/l,3.5-5.1,08.01.18 11:15:00,5.84,meq/l,E87.5,5.84mmol/l > 5.5mmol/l,NaN,NaN,NaN,NaN
3,2317025683,Glucose,7.27,mmol/l,73-107,10.01.18 10:16:00,131.00,NaN,E11.91,7.27mmol/l > 5.0mmol/l,NaN,NaN,NaN,NaN
4,2317025683,Glucose,10.43,mmol/l,73-107,12.01.18 16:21:00,188.00,NaN,E11.91,10.43mmol/l > 5.0mmol/l,NaN,NaN,NaN,NaN


In [101]:
alle["Info zu Prüfgrund"] = alle["Laborkürzel"].astype(str) + ": " + alle["Info zu Prüfgrund"].astype(str).loc[alle["Info zu Prüfgrund"].notnull()]
alle["Info zu Prüfgrund 2"] = alle["ICD Klarname"].astype(str) + "gef. in: " + alle["ganzer Befund"].astype(str).loc[alle["ICD Klarname"].notnull()]
alle.drop(labels=["Laborkürzel","Messwert","Masseinheit","Referenzbereich","alt. Messwert", "alt. Masseinheit","ICD Klarname","Fundstelle im Text","ganzer Befund"],axis=1,inplace=True)
alle.rename(columns={"ICD":"Prüfgrund 2",},index=str,inplace=True)
print(len(alle))
alle.head()

60319


,Fallnummer,Datum,Prüfgrund,Info zu Prüfgrund,Prüfgrund 2,Info zu Prüfgrund 2
0,2317025683,05.01.18 08:18:00,E11.91,Glucose: 14.15mmol/l > 5.0mmol/l,NaN,NaN
1,2317025683,08.01.18 11:15:00,E11.91,Glucose: 14.87mmol/l > 5.0mmol/l,NaN,NaN
2,2317025683,08.01.18 11:15:00,E87.5,Kalium: 5.84mmol/l > 5.5mmol/l,NaN,NaN
3,2317025683,10.01.18 10:16:00,E11.91,Glucose: 7.27mmol/l > 5.0mmol/l,NaN,NaN
4,2317025683,12.01.18 16:21:00,E11.91,Glucose: 10.43mmol/l > 5.0mmol/l,NaN,NaN


In [102]:
icd = pd.read_csv("ICD.csv",sep=";",encoding="latin")
print(len(icd))
icd.head()

96401


,IK,Entlassender-Standort,Entgeltbereich,KH-internes-Kennzeichen,Diagnoseart,ICD-Version,ICD-Kode,Lokalisation,Diagnosensicherheit,Sekundär-Kode,Lokalisation.1,Diagnosensicherheit.1
0,260200206,NaN,DRG,2317041813,HD,2018,E04.2,NaN,NaN,NaN,NaN,NaN
1,260200206,NaN,DRG,2317041813,ND,2018,R11,NaN,NaN,NaN,NaN,NaN
2,260200206,NaN,DRG,2317041813,ND,2018,E04.2,NaN,NaN,NaN,NaN,NaN
3,260200206,NaN,DRG,2317039149,HD,2017,K62.4,NaN,NaN,NaN,NaN,NaN
4,260200206,NaN,DRG,2317039149,ND,2017,K43.3,NaN,NaN,NaN,NaN,NaN


In [103]:
icd = icd.groupby('KH-internes-Kennzeichen')['ICD-Kode'].apply(', '.join).reset_index()

In [104]:
drg = pd.read_csv("ENTGELTE.csv",sep=";",encoding="latin")
drg.head()

,IK,Entlassender-Standort,Entgeltbereich,KH-internes-Kennzeichen,IK-Krankenkasse,Entgeltart,Entgeltbetrag,Abrechnung-von,Abrechnung-bis,Entgeltanzahl,Tage-ohne-Berechnung-Behandlung,Tag-der-Behandlung
0,260200206,NaN,DRG,2317041813,101580004.0,00000000,"3306,92",NaN,NaN,1,0.0,NaN
1,260200206,NaN,DRG,2317041813,101580004.0,46002000,"1,16",20180111.0,20180112.0,1,NaN,NaN
2,260200206,NaN,DRG,2317041813,101580004.0,48000001,"1,31",20180111.0,20180112.0,1,NaN,NaN
3,260200206,NaN,DRG,2317041813,101580004.0,47100001,"1,70",20180111.0,20180112.0,1,NaN,NaN
4,260200206,NaN,DRG,2317041813,101580004.0,47100012,"9,73",20180111.0,20180112.0,1,NaN,NaN


In [105]:
drg = drg.loc[drg["Entgeltart"].str.contains("\D")]
drg = drg[["KH-internes-Kennzeichen","Entgeltart", "Entgeltbetrag"]]
drg.head()

,KH-internes-Kennzeichen,Entgeltart,Entgeltbetrag
8,2317041813,7010K06D,"3242,80"
14,2317039149,76ZED800,"7506,10"
15,2317039149,7010G35Z,"34009,20"
24,2317039153,7310F71B,"840,85"
25,2317039153,7010F71B,"1527,60"


In [106]:
p21 = icd.merge(drg, how="outer", on="KH-internes-Kennzeichen")

In [107]:
p21.rename(columns={"KH-internes-Kennzeichen":"Fallnummer",},index=str,inplace=True)
p21.head()

,Fallnummer,ICD-Kode,Entgeltart,Entgeltbetrag
0,2317025683,"D37.5, A41.51, I21.4, K25.4, K61.1, T81.3, J18...",76ZEA101,"100,31"
1,2317025683,"D37.5, A41.51, I21.4, K25.4, K61.1, T81.3, J18...",76ZE0101,"225,21"
2,2317025683,"D37.5, A41.51, I21.4, K25.4, K61.1, T81.3, J18...",76ZED001,"1004,49"
3,2317025683,"D37.5, A41.51, I21.4, K25.4, K61.1, T81.3, J18...",7110A06A,"1711,85"
4,2317025683,"D37.5, A41.51, I21.4, K25.4, K61.1, T81.3, J18...",76ZEA706,"3539,24"


In [108]:
fallliste = alle.merge(p21, how="left", on="Fallnummer")

In [109]:
fallliste

,Fallnummer,Datum,Prüfgrund,Info zu Prüfgrund,Prüfgrund 2,Info zu Prüfgrund 2,ICD-Kode,Entgeltart,Entgeltbetrag
0,2317025683,05.01.18 08:18:00,E11.91,Glucose: 14.15mmol/l > 5.0mmol/l,NaN,NaN,"D37.5, A41.51, I21.4, K25.4, K61.1, T81.3, J18...",76ZEA101,"100,31"
1,2317025683,05.01.18 08:18:00,E11.91,Glucose: 14.15mmol/l > 5.0mmol/l,NaN,NaN,"D37.5, A41.51, I21.4, K25.4, K61.1, T81.3, J18...",76ZE0101,"225,21"
2,2317025683,05.01.18 08:18:00,E11.91,Glucose: 14.15mmol/l > 5.0mmol/l,NaN,NaN,"D37.5, A41.51, I21.4, K25.4, K61.1, T81.3, J18...",76ZED001,"1004,49"
3,2317025683,05.01.18 08:18:00,E11.91,Glucose: 14.15mmol/l > 5.0mmol/l,NaN,NaN,"D37.5, A41.51, I21.4, K25.4, K61.1, T81.3, J18...",7110A06A,"1711,85"
4,2317025683,05.01.18 08:18:00,E11.91,Glucose: 14.15mmol/l > 5.0mmol/l,NaN,NaN,"D37.5, A41.51, I21.4, K25.4, K61.1, T81.3, J18...",76ZEA706,"3539,24"
5,2317025683,05.01.18 08:18:00,E11.91,Glucose: 14.15mmol/l > 5.0mmol/l,NaN,NaN,"D37.5, A41.51, I21.4, K25.4, K61.1, T81.3, J18...",7010A06A,"207338,20"
6,2317025683,08.01.18 11:15:00,E11.91,Glucose: 14.87mmol/l > 5.0mmol/l,NaN,NaN,"D37.5, A41.51, I21.4, K25.4, K61.1, T81.3, J18...",76ZEA101,"100,31"
7,2317025683,08.01.18 11:15:00,E11.91,Glucose: 14.87mmol/l > 5.0mmol/l,NaN,NaN,"D37.5, A41.51, I21.4, K25.4, K61.1, T81.3, J18...",76ZE0101,"225,21"
8,2317025683,08.01.18 11:15:00,E11.91,Glucose: 14.87mmol/l > 5.0mmol/l,NaN,NaN,"D37.5, A41.51, I21.4, K25.4, K61.1, T81.3, J18...",76ZED001,"1004,49"
9,2317025683,08.01.18 11:15:00,E11.91,Glucose: 14.87mmol/l > 5.0mmol/l,NaN,NaN,"D37.5, A41.51, I21.4, K25.4, K61.1, T81.3, J18...",7110A06A,"1711,85"


In [135]:
quellen = ["MIBI"]
quellen_dateien = ["mibi/mibi_test_synonyme/textsuche_ergebnisse_2019-03-06_19-02-43.csv"]
quelle_lab = pd.read_csv("labor/Labor_2018_mit_Pruefgrund.csv",encoding="latin1",sep=";")

quellen_dfs =  [pd.read_csv(q,encoding="latin1",sep=";") for q in quellen_dateien]
quellen_dfs.append(quelle_lab)

In [50]:
new_fallliste = pd.DataFrame(columns=fallliste.columns)
for khid in fallnr_pruefgrund.keys():
    id_row = fallliste.iloc[fallnr_min_diff_idx[khid],:].copy()
    for i,pruefgrund in enumerate(fallnr_pruefgrund[khid]):
        id_row["Pruefgrund "+str(i+1)] = pruefgrund
        found_in_text = False
        for q_df,name in zip(quellen_dfs,quellen):
            nachweis = q_df[(q_df["Fallnr."]==khid)&(q_df["ICD"].str.contains(pruefgrund))]
            #if len(nachweis.iloc[:,0])>1:
                #print(nachweis)
            if not nachweis.empty:
                #print(nachweis["ICD Klarame"])
                if "Nachweis "+str(i+1) in id_row.index:
                    id_row["Nachweis "+str(i+1)] = id_row["Nachweis "+str(i+1)] + nachweis["ICD Klarname"].unique()[0] + " gefunden in {}: ".format(name) +" | ".join(nachweis["Fundstelle im Text"].str.replace("\n","").str.replace("\r","").values)
                else:
                    id_row["Nachweis "+str(i+1)] = nachweis["ICD Klarname"].unique()[0] + " gefunden in {}: ".format(name) +" | ".join(nachweis["Fundstelle im Text"].str.replace("\n","").str.replace("\r","").values)

        nachweis = quelle_lab[(quelle_lab["KH-ID"]==khid)&(quelle_lab["Prüfgrund"].str.contains(pruefgrund))]
        if not nachweis.empty:# and len(nachweis.iloc[:,0])<2:
            #print(nachweis)
            if "Nachweis "+str(i+1) in id_row.index:
                a=" am "
                for x,y,z in zip(nachweis["Datum"],nachweis["Messwert"],nachweis["Masseinheit"]):
                    a += str(x)[:8] + " = "+str(y)+" " + str(z) + " | "
                id_row["Nachweis "+str(i+1)] = id_row["Nachweis "+str(i+1)] + " | " + pruefgrund + " da in LABOR "+ str(nachweis["Laborkürzel"].unique()[0]) + a
         
            else: 
                a=" am "
                for x,y,z in zip(nachweis["Datum"],nachweis["Messwert"],nachweis["Masseinheit"]):
                    a += str(x)[:8] +" = " + str(y) +  " "+str(z)+" | "
                id_row["Nachweis "+str(i+1)] =  pruefgrund + " da in LABOR " + str(nachweis["Laborkürzel"].unique()[0]) + a

    new_fallliste = new_fallliste.append(id_row)
    

In [51]:
cols = new_fallliste.columns.tolist()
print(cols)

['HDL_Fallnr', 'Fallnr', 'MA', 'Kommentar ', 'Pr\x81fgrund', 'Unnamed: 5', 'Unnamed: 6', 'Unnamed: 7', 'Unnamed: 8', 'Unnamed: 9', 'Unnamed: 10', 'SZ', 'MDK?', 'MDK-Pr\x81fgrund', 'DRG_aus_SIM', 'DRG aus Entgelte', 'Diff', 'MDC', 'PCCL', 'Gruppierungsstatus', 'BWR_neu', 'BWR alt (aus SIM_pur)', 'Diff BWR', 'uGVD', 'mVWD', 'oVGD', 'VWD', 'Entgelt', 'Entgeltschl\x81ssel', 'Entgeltanzahl', 'Datum Abrechnung', 'Datum2', 'Entgelt Gesamtbetrag', 'Entlassungsgrund', 'FA-Typ', 'Unnamed: 35', 'Unnamed: 36', 'Unnamed: 37', 'Unnamed: 38', 'Unnamed: 39', 'Unnamed: 40', 'Unnamed: 41', 'Unnamed: 42', 'Unnamed: 43', 'Unnamed: 44', 'Unnamed: 45', 'Unnamed: 46', 'Unnamed: 47', 'Unnamed: 48', 'Unnamed: 49', 'Unnamed: 50', 'Unnamed: 51', 'Unnamed: 52', 'Unnamed: 53', 'Unnamed: 54', 'Unnamed: 55', 'Unnamed: 56', 'Unnamed: 57', 'Unnamed: 58', 'Unnamed: 59', 'Unnamed: 60', 'Unnamed: 61', 'Unnamed: 62', 'Unnamed: 63', 'Unnamed: 64', 'Unnamed: 65', 'Unnamed: 66', 'Unnamed: 67', 'Unnamed: 68', 'Unnamed: 69', '

In [61]:
cols = ['HDL_Fallnr', 'Fallnr', 'MA', 'Kommentar ', 'Pr\x81fgrund', 'Unnamed: 5', 'Unnamed: 6', 'Unnamed: 7', 'Unnamed: 8', 'Unnamed: 9', 'Unnamed: 10', 'SZ', 'MDK?', 'MDK-Pr\x81fgrund', 'DRG_aus_SIM', 'DRG aus Entgelte', 'Diff', 'MDC', 'PCCL', 'Gruppierungsstatus', 'BWR_neu', 'BWR alt (aus SIM_pur)', 'Diff BWR', 'uGVD', 'mVWD', 'oVGD', 'VWD', 'Entgelt', 'Entgeltschl\x81ssel', 'Entgeltanzahl', 'Datum Abrechnung', 'Datum2', 'Entgelt Gesamtbetrag', 'Entlassungsgrund', 'FA-Typ', 'Unnamed: 35', 'Unnamed: 36', 'Unnamed: 37', 'Unnamed: 38', 'Unnamed: 39', 'Unnamed: 40', 'Unnamed: 41', 'Unnamed: 42', 'Unnamed: 43', 'Unnamed: 44', 'Unnamed: 45', 'Unnamed: 46', 'Unnamed: 47', 'Unnamed: 48', 'Unnamed: 49', 'Unnamed: 50', 'Unnamed: 51', 'Unnamed: 52', 'Unnamed: 53', 'Unnamed: 54', 'Unnamed: 55', 'Unnamed: 56', 'Unnamed: 57', 'Unnamed: 58', 'Unnamed: 59', 'Unnamed: 60', 'Unnamed: 61', 'Unnamed: 62', 'Unnamed: 63', 'Unnamed: 64', 'Unnamed: 65', 'Unnamed: 66', 'Unnamed: 67', 'Unnamed: 68', 'Unnamed: 69', 'Nachweis 1','Nachweis 2', 'Nachweis 3','Pruefgrund 1',"Pruefgrund 2",    'Pruefgrund 3']

In [62]:
new_fallliste = new_fallliste[cols]
new_fallliste.to_csv("Ergebnissliste.csv",encoding="latin1",sep=";",index=False)

In [59]:
id_row.index

Index(['HDL_Fallnr', 'Fallnr', 'MA', 'Kommentar ', 'Prfgrund', 'Unnamed: 5',
       'Unnamed: 6', 'Unnamed: 7', 'Unnamed: 8', 'Unnamed: 9', 'Unnamed: 10',
       'SZ', 'MDK?', 'MDK-Prfgrund', 'DRG_aus_SIM', 'DRG aus Entgelte',
       'Diff', 'MDC', 'PCCL', 'Gruppierungsstatus', 'BWR_neu',
       'BWR alt (aus SIM_pur)', 'Diff BWR', 'uGVD', 'mVWD', 'oVGD', 'VWD',
       'Entgelt', 'Entgeltschlssel', 'Entgeltanzahl', 'Datum Abrechnung',
       'Datum2', 'Entgelt Gesamtbetrag', 'Entlassungsgrund', 'FA-Typ',
       'Unnamed: 35', 'Unnamed: 36', 'Unnamed: 37', 'Unnamed: 38',
       'Unnamed: 39', 'Unnamed: 40', 'Unnamed: 41', 'Unnamed: 42',
       'Unnamed: 43', 'Unnamed: 44', 'Unnamed: 45', 'Unnamed: 46',
       'Unnamed: 47', 'Unnamed: 48', 'Unnamed: 49', 'Unnamed: 50',
       'Unnamed: 51', 'Unnamed: 52', 'Unnamed: 53', 'Unnamed: 54',
       'Unnamed: 55', 'Unnamed: 56', 'Unnamed: 57', 'Unnamed: 58',
       'Unnamed: 59', 'Unnamed: 60', 'Unnamed: 61', 'Unnamed: 62',
       'Unnamed: 6

In [55]:
new_fallliste

,HDL_Fallnr,Fallnr,MA,Kommentar,Prfgrund,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,...,Unnamed: 66,Unnamed: 67,Unnamed: 68,Unnamed: 69,Nachweis 1,Pruefgrund 1,Nachweis 2,Pruefgrund 2,Nachweis 3,Pruefgrund 3
51,NaN,1755554917,NaN,NaN,SIM_R18,NaN,NaN,NaN,NaN,NaN,...,NaN,20180103,NaN,NaN,aszites gefunden in RIS: neu aufgetretene klei...,R18,NaN,NaN,NaN,NaN
159,NaN,1756820617,NaN,NaN,SIM_J93.0,NaN,NaN,NaN,NaN,NaN,...,NaN,20180104,NaN,NaN,pneumotho gefunden in RIS: thorax a.p.(intensi...,J93.0,NaN,NaN,NaN,NaN
245,NaN,1757195217,NaN,NaN,SIM_J90,NaN,NaN,NaN,NaN,NaN,...,NaN,20180107,20180107.0,NaN,pleuraerg gefunden in RIS: pleuraerguss links....,J90,aszites gefunden in RIS: intraabdominell gerin...,R18,NaN,NaN
9,NaN,1757839718,NaN,NaN,SIM_J90,NaN,NaN,NaN,NaN,NaN,...,NaN,20180117,NaN,NaN,pleuraerg gefunden in RIS: altersentsprechende...,J90,NaN,NaN,NaN,NaN
49,NaN,1757927517,NaN,NaN,SIM_K56.0,NaN,NaN,NaN,NaN,NaN,...,NaN,20180130,NaN,NaN,peritonitis gefunden in PATHO: kein hinweis au...,K56.0,NaN,NaN,NaN,NaN
190,NaN,1758790417,NaN,NaN,SIM_J90,NaN,NaN,NaN,NaN,NaN,...,NaN,20180111,NaN,NaN,pleuraerg gefunden in RIS: orientierend keine ...,J90,NaN,NaN,NaN,NaN
189,NaN,1759132517,NaN,NaN,SIM_B95.7,NaN,NaN,NaN,NaN,NaN,...,NaN,20180115,NaN,NaN,b95.7 gefunden in MIBI: hominis punktat in ...,B95.7,NaN,NaN,NaN,NaN
114,NaN,1759269617,NaN,NaN,SIM_J90,NaN,NaN,NaN,NaN,NaN,...,NaN,20180111,NaN,NaN,pleuraerg gefunden in RIS: im lungenfenster ze...,J90,atelektase gefunden in RIS: mit kompressionsat...,J98.1,NaN,NaN
243,NaN,1759294418,NaN,NaN,SIM_J18.9,NaN,NaN,NaN,NaN,NaN,...,NaN,20180307,NaN,NaN,pneumonie gefunden in RIS: vom aspekt her über...,J18.9,NaN,NaN,NaN,NaN
208,NaN,1759476617,NaN,NaN,SIM_J90,NaN,NaN,NaN,NaN,NaN,...,NaN,20180107,20180107.0,NaN,pleuraerg gefunden in RIS: nebenbefundlich ger...,J90,NaN,NaN,NaN,NaN


In [20]:
a = "11"
a +="343"
a

'11343'